In [4]:
import pandas as pd

from watcher import Watcher, Chain
from constants import FUCK_ADDRESS

watcher = Watcher(chain=Chain.SOL)
tokens_invested_response = watcher.grab_tokens_invested(FUCK_ADDRESS)

In [5]:
df = pd.DataFrame(tokens_invested_response.data["holdings"])

In [8]:
df.columns

Index(['address', 'token_address', 'symbol', 'name', 'decimals', 'logo',
       'balance', 'usd_value', 'realized_profit_30d', 'realized_profit',
       'realized_pnl', 'realized_pnl_30d', 'unrealized_profit',
       'unrealized_pnl', 'total_profit', 'total_profit_pnl', 'avg_cost',
       'avg_sold', 'buy_30d', 'sell_30d', 'sells', 'price', 'cost',
       'position_percent', 'last_active_timestamp', 'history_sold_income',
       'history_bought_cost', 'price_change_5m', 'price_change_1h',
       'price_change_6h', 'price_change_24h', 'is_following', 'is_show_alert',
       'is_honeypot'],
      dtype='object')

In [15]:
df[[
    "token_address",
    "symbol",
    "name",
    "realized_profit",
    "cost",
    "history_bought_cost",
    "history_sold_income",
]].head()

,token_address,symbol,name,realized_profit,cost,history_bought_cost,history_sold_income
0,52ScRbUR7y8AVqkMjE29FJVjDPELSPofJwaXaLHepump,TROLL,Troll by Matt Furie,-3.275838,0.743575,7.43575,3.416337
1,8CUbzzN5Q9Tnww8ciDTdcCPXVmi8mymAApot9Hyspump,gloop,Gloop By Matt Furie,-3.778678,0.743175,7.43175,2.909897
2,Fabws3AB7BAowYDywX8Gnb8QBF13TSpSpv3FHUS6pump,SHADES,SHADECREEPER BY MATT FURIE,-3.398939,0.742525,7.42525,3.283786
3,FLJy5tBAxHnzAC3bodjRAtSxoEGmwniqbgQ9LE7Ppump,BILZ,Dan Bilzerian,-4.386256,0.743525,7.43525,2.305469
4,84mypPSjeNtXGRe38xJoMrfEfehvu7X8rBSEoDLRbUb1,PEPEZO,ZOGGED PEPE BY MATT FURIE,-5.585620,0.738575,7.38575,1.061555
